In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from anthropic import AnthropicBedrock
from application import Application
from compiler.core import Compiler
from core.interpolator import Interpolator

In [4]:
compiler = Compiler("botbuild/tsp_compiler", "botbuild/app_schema")
client = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

In [5]:
application = Application(client, compiler)
interpolator = Interpolator(".")

In [6]:
application_description = """
Bot that does web search
""".strip()

In [7]:
application_description = "Generate a bot that stores and searches notes"

In [9]:
my_bot = application.create_bot(application_description, bot_id="handler_tests")

Skipping Initial Description Refinement
Compiling TypeSpec...


Exception: Failed to generate typespec: TypeSpec compiler v0.64.0

Diagnostics were reported during compilation:

schema.tsp:5:12 - error missing-implementation: Extern declaration must have an implementation in JS file.
> 5 | extern dec scenario(target: unknown, description: string);
    |            ^^^^^^^^

Found 1 error.



In [9]:
interpolator.bake(my_bot, "../../bot_result") # next to repo root

In [10]:
my_bot.__dict__

{'refined_description': RefineOut(refined_description='Generate a bot that stores and searches notes', error_output=None),
 'typespec': TypespecOut(reasoning="The errors are due to missing 'id' parameter in @llm_func decorator. I'll add unique ids for each function.\nAlso verified that no TypeSpec reserved keywords (model, interface) are used in:\n- Property names: all properties like 'id', 'title', 'content', etc. are valid\n- Type names: Note, SaveNoteRequest, SearchNotesRequest, DeleteNoteRequest are valid\n- Function names: saveNote, searchNotes, deleteNote are valid", typespec_definitions='model Note {\n    id: string;\n    title: string;\n    content: string;\n    tags: string[];\n    createdAt: utcDateTime;\n}\n\nmodel SaveNoteRequest {\n    title: string;\n    content: string;\n    tags: string[];\n}\n\nmodel SearchNotesRequest {\n    query?: string;\n    from?: utcDateTime;\n    to?: utcDateTime;\n    tags?: string[];\n}\n\nmodel DeleteNoteRequest {\n    id: string;\n}\n\ninte